# Q2

In [1]:
import pandas as pd
import numpy as np
import csv
from collections import defaultdict
from sklearn.preprocessing import KBinsDiscretizer

In [2]:
class RationalCategorization:
    """
    Adapted Anderson's Rational Model for Supervised Categorization.
    """

    def __init__(self, alpha=1.0, n_bins=3):
        """
        Initialize the model.

        Args:
            alpha (float): Smoothing parameter for probability estimation.
            n_bins (int): Number of bins to discretize continuous features.
        """
        self.alpha = alpha
        self.n_bins = n_bins
        self.categories = {}
        self.feature_bins = {}
        self.feature_probs = {}
        self.category_counts = defaultdict(int)
        self.total_count = 0

    def discretize_features(self, X):
        """
        Discretize continuous features into categorical bins.

        Args:
            X (np.ndarray): Continuous feature matrix.

        Returns:
            np.ndarray: Discretized feature matrix.
        """
        self.discretizer = KBinsDiscretizer(n_bins=self.n_bins, encode='ordinal', strategy='uniform')
        X_discrete = self.discretizer.fit_transform(X)
        return X_discrete.astype(int)

    def fit(self, X, y):
        """
        Train the model with labeled data.

        Args:
            X (np.ndarray): Continuous feature matrix.
            y (np.ndarray): Category labels.
        """
        # Discretize features
        X_discrete = self.discretize_features(X)

        self.total_count = len(y)
        categories = np.unique(y)
        self.categories = {cat: idx for idx, cat in enumerate(categories)}  # Map category to index

        n_features = X_discrete.shape[1]
        n_categories = len(categories)

        # Initialize feature probability dictionaries
        self.feature_probs = {cat: [defaultdict(lambda: self.alpha) for _ in range(n_features)] for cat in categories}

        # Count feature occurrences per category
        for xi, yi in zip(X_discrete, y):
            self.category_counts[yi] += 1
            for i, val in enumerate(xi):
                self.feature_probs[yi][i][val] += 1

        # Convert counts to probabilities
        for cat in categories:
            for i in range(n_features):
                total = self.category_counts[cat] * self.n_bins * self.alpha
                for val in range(self.n_bins):
                    self.feature_probs[cat][i][val] /= (self.category_counts[cat] + self.n_bins * self.alpha)

    def predict(self, X):
        """
        Predict categories for given data.

        Args:
            X (np.ndarray): Continuous feature matrix.

        Returns:
            List: Predicted category labels.
        """
        X_discrete = self.discretizer.transform(X).astype(int)
        predictions = []

        for xi in X_discrete:
            category_scores = {}
            for cat in self.categories:
                # Calculate log-probabilities to avoid underflow
                log_prob = np.log(self.category_counts[cat] / self.total_count)
                for i, val in enumerate(xi):
                    log_prob += np.log(self.feature_probs[cat][i].get(val, self.alpha / (self.category_counts[cat] + self.n_bins * self.alpha)))
                category_scores[cat] = log_prob
            # Select category with highest probability
            predicted_category = max(category_scores, key=category_scores.get)
            predictions.append(predicted_category)
        return predictions

def load_csv_features_labels(filepath):
    """
    Load features and labels from a CSV file.

    Args:
        filepath (str): Path to the CSV file.

    Returns:
        Tuple[np.ndarray, np.ndarray]: Features and labels.
    """
    X = []
    y = []
    with open(filepath, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            # Assuming the CSV has no header
            weight = float(row[0])
            height = float(row[1])
            label = int(row[2])
            X.append([weight, height])
            y.append(label)
    return np.array(X), np.array(y)

def load_csv_features(filepath):
    """
    Load features from a CSV file.

    Args:
        filepath (str): Path to the CSV file.

    Returns:
        np.ndarray: Features.
    """
    X = []
    with open(filepath, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            weight = float(row[0])
            height = float(row[1])
            X.append([weight, height])
    return np.array(X)


In [4]:
def main():
    # Paths to your CSV files
    train_filepath = 'X.csv'
    test_filepath = 'y.csv'

    # Load training data
    X_train, y_train = load_csv_features_labels(train_filepath)

    # Initialize and train the model
    model = RationalCategorization(alpha=1.0, n_bins=3)  # You can adjust n_bins as needed
    model.fit(X_train, y_train)

    # Load test data
    X_test = load_csv_features(test_filepath)

    # Predict categories for test data
    predictions = model.predict(X_test)

    # Output the predictions
    print("Test Set Predictions:")
    for i, (stimulus, pred) in enumerate(zip(X_test, predictions), 1):
        weight, height = stimulus
        category = {1: 'Small', 2: 'Average', 3: 'Large'}.get(pred, 'Unknown')
        print(f"Data Point {i}: Weight = {weight} kg, Height = {height} inches -> Predicted Category: {category} ({pred})")

if __name__ == '__main__':
    main()


Test Set Predictions:
Data Point 1: Weight = 74.0 kg, Height = 67.0 inches -> Predicted Category: Large (3)
Data Point 2: Weight = 69.0 kg, Height = 63.0 inches -> Predicted Category: Average (2)
Data Point 3: Weight = 92.0 kg, Height = 81.0 inches -> Predicted Category: Large (3)
Data Point 4: Weight = 64.0 kg, Height = 61.0 inches -> Predicted Category: Average (2)
Data Point 5: Weight = 66.0 kg, Height = 84.0 inches -> Predicted Category: Average (2)
Data Point 6: Weight = 76.0 kg, Height = 68.0 inches -> Predicted Category: Large (3)
Data Point 7: Weight = 61.0 kg, Height = 58.0 inches -> Predicted Category: Average (2)
Data Point 8: Weight = 64.0 kg, Height = 76.0 inches -> Predicted Category: Average (2)
Data Point 9: Weight = 68.0 kg, Height = 66.0 inches -> Predicted Category: Average (2)
Data Point 10: Weight = 34.0 kg, Height = 61.0 inches -> Predicted Category: Small (1)


# Q3) for RMC

In [5]:
# Load the data from X.csv
data = pd.read_csv("X.csv", header=None)

# Shuffle the rows randomly
shuffled_data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the shuffled data to a new CSV file
shuffled_data.to_csv("X_shuffled.csv", index=False, header=False)

print("Data has been shuffled and saved as X_shuffled.csv")

Data has been shuffled and saved as X_shuffled.csv


In [6]:
def main():
    # Paths to your CSV files
    train_filepath = 'X_shuffled.csv'
    test_filepath = 'y.csv'

    # Load training data
    X_train, y_train = load_csv_features_labels(train_filepath)

    # Initialize and train the model
    model = RationalCategorization(alpha=1.0, n_bins=3)  # You can adjust n_bins as needed
    model.fit(X_train, y_train)

    # Load test data
    X_test = load_csv_features(test_filepath)

    # Predict categories for test data
    predictions = model.predict(X_test)

    # Output the predictions
    print("Test Set Predictions:")
    for i, (stimulus, pred) in enumerate(zip(X_test, predictions), 1):
        weight, height = stimulus
        category = {1: 'Small', 2: 'Average', 3: 'Large'}.get(pred, 'Unknown')
        print(f"Data Point {i}: Weight = {weight} kg, Height = {height} inches -> Predicted Category: {category} ({pred})")

if __name__ == '__main__':
    main()

Test Set Predictions:
Data Point 1: Weight = 74.0 kg, Height = 67.0 inches -> Predicted Category: Large (3)
Data Point 2: Weight = 69.0 kg, Height = 63.0 inches -> Predicted Category: Average (2)
Data Point 3: Weight = 92.0 kg, Height = 81.0 inches -> Predicted Category: Large (3)
Data Point 4: Weight = 64.0 kg, Height = 61.0 inches -> Predicted Category: Average (2)
Data Point 5: Weight = 66.0 kg, Height = 84.0 inches -> Predicted Category: Average (2)
Data Point 6: Weight = 76.0 kg, Height = 68.0 inches -> Predicted Category: Large (3)
Data Point 7: Weight = 61.0 kg, Height = 58.0 inches -> Predicted Category: Average (2)
Data Point 8: Weight = 64.0 kg, Height = 76.0 inches -> Predicted Category: Average (2)
Data Point 9: Weight = 68.0 kg, Height = 66.0 inches -> Predicted Category: Average (2)
Data Point 10: Weight = 34.0 kg, Height = 61.0 inches -> Predicted Category: Small (1)
